# Getting Data from Google Earth Engine

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import folium
import geemap.core as geemap
import geehydro
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.svm import LinearSVC
# from sklearn import svm, metrics, ensemble
# from keras.models import Sequential, Model
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, MaxPool2D
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.utils import plot_model

# from tensorflow.keras.optimizers import Adam
# from keras.callbacks import LearningRateScheduler
# from keras.regularizers import l2, l1
import math
import requests
import os
import skimage



### Authenticate and Initialize Project

In [2]:
try:
## should be a one-time run
    ee.Authenticate()
    ee.Initialize(project="pipeline-elevation-project")
except:
    ee.Initialize(project="pipeline-elevation-project")

ConnectionError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000023A6D00DD10>: Failed to resolve 'oauth2.googleapis.com' ([Errno 11001] getaddrinfo failed)"))

### Call Landsat Data

In [8]:
# filter boundary:
tx = ee.Geometry.Rectangle(-106.64719063660635,25.840437651866516,-93.5175532104321,36.50050935248352)

i_date = '2022-05-01'
f_date = '2022-06-30'

In [4]:
lstColl = ee.ImageCollection("LANDSAT/LC09/C02/T1")

lst = lstColl.select('B1','B2','B3','B4').filterDate(i_date,f_date).filterBounds(tx)

In [23]:
img = lst.toList(100).get(25)
display(img)

##### get known pipeline locations for coordinates

* [tx_ngpipeline data](https://drive.google.com/drive/u/0/folders/1kSXfSin5b7wr6S_LMrRoc4sobY-UC-Qq)

In [11]:
zipPath = "D:/Downloads/"
ngPipePath = zipPath+"TX_NGPipe/NaturalGas_Pipelines_TX.shp"
ngPipes = gpd.read_file(ngPipePath)
MercNGPipes = ngPipes.to_crs(epsg=4326)
MercNGPipes["centroids"] = MercNGPipes.geometry.centroid
MercNGPipes["beg"] = MercNGPipes.geometry.boundary.centroid



C:\Users\timol\AppData\Local\Temp\ipykernel_28368\2156654528.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  MercNGPipes["centroids"] = MercNGPipes.geometry.centroid
C:\Users\timol\AppData\Local\Temp\ipykernel_28368\2156654528.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  MercNGPipes["beg"] = MercNGPipes.geometry.boundary.centroid


## get bounding box, picture, and all

In [33]:
def mapImg(img,lon,lat):
    Map = folium.Map()
    Map.setOptions()

    Map.setCenter(lon,lat) # coordinates of poi
    # Map.addLayer(tx, {'color': '#FECA1E', 'fillColor': '#4c4cff'})
    Map.addLayer(img, {}, 'default color composite')


    Map.setControlVisibility()
    return Map

### Section takes a while to run, but gets appropriate pictures (pipeline 835 was first success)

In [49]:
DEBUG = True
metersPerUnit = (1.11/0.00001)
FHmeters = 500/metersPerUnit
aerColl = ee.ImageCollection("USDA/NAIP/DOQQ")
orthColl = ee.ImageCollection("SKYSAT/GEN-A/PUBLIC/ORTHO/RGB")
# tx = ee.Geometry.Rectangle(-106.64719063660635,25.840437651866516,-93.5175532104321,36.50050935248352)
i_date = '2022-05-01'
f_date = '2022-06-30'


for i in range(len(MercNGPipes)):
    # using new shapefile, so more 
    lat = MercNGPipes.centroids[i:i+1].squeeze().y
    lon = MercNGPipes.centroids[i:i+1].squeeze().x
    # bbox args: west, south, east, north
    bbox = ee.Geometry.BBox(lon-FHmeters,lat-FHmeters,lon+FHmeters,lat+FHmeters)

    # get image(s) within box
    aList = aerColl.select('R','G','B','N').filterDate(i_date,f_date).filterBounds(bbox)
    oList = orthColl.select('R','G','B').filterDate(i_date,f_date).filterBounds(bbox)
    ## need to check if list is empty
    ## use .filterBounds(tx) if later issues

    aImg = ee.Image(aList.toList(100).get(1))
    try:
        mapImg(aImg,lon,lat)
        ## Export section
        task = ee.batch.Export.image.toCloudStorage(aImg,
                                description=f"USDA_aerials/pipeline{i}",  # put part of pipeline here too
                                # ^ put desired name for the task & file in cloud storage
                                bucket = "gee_image_exports",  # should be gee_image_exports
                                region=bbox,  # I wonder if filtering by box first works too
                                #maxPixels=1500000000  
                                )
        task.start()
        ##### to view proper map
        print(f"Aerial Ag {i}")
    except:
        if DEBUG:
            print(f"skipped agriculture pipe {i}")
        pass

    oImg = ee.Image(oList.toList(100).get(1))
    try:
        mapImg(oImg,lon,lat)
        ## Export section
        task = ee.batch.Export.image.toCloudStorage(oImg,
                                description=f"SKYSAT_ortho/pipeline{i}",  # put part of pipeline here too
                                # ^ put desired name for the task & file in cloud storage
                                bucket = "gee_image_exports",  # should be gee_image_exports
                                region=bbox,  # I wonder if filtering by box first works too
                                #maxPixels=1500000000  
                                )
        task.start()
        ##### to view proper map
        print(f"Aerial Orthro {i}")
        continue
    except:
        if DEBUG:
            print(f"skipped ortho pipe {i}")
        pass

Aerial Ag 835
Aerial Ag 836
Aerial Ag 838
Aerial Ag 839
Aerial Ag 843
Aerial Ag 847
Aerial Ag 849
Aerial Ag 851
Aerial Ag 853
Aerial Ag 860
Aerial Ag 861
Aerial Ag 862
Aerial Ag 863
Aerial Ag 864
Aerial Ag 865
Aerial Ag 872
Aerial Ag 877
Aerial Ag 894
Aerial Ag 895
Aerial Ag 896
Aerial Ag 898
Aerial Ag 902
Aerial Ag 903
Aerial Ag 904
Aerial Ag 906
Aerial Ag 907
Aerial Ag 908
Aerial Ag 912
Aerial Ag 914
Aerial Ag 915
Aerial Ag 916
Aerial Ag 917
Aerial Ag 920
Aerial Ag 922
Aerial Ag 923
Aerial Ag 924
Aerial Ag 927
Aerial Ag 928
Aerial Ag 934
Aerial Ag 935
Aerial Ag 936
Aerial Ag 995
Aerial Ag 1001
Aerial Ag 1002
Aerial Ag 1004
Aerial Ag 1005
Aerial Ag 1017
Aerial Ag 1019
Aerial Ag 1024
Aerial Ag 1069
Aerial Ag 1088
Aerial Ag 1095
Aerial Ag 1097
Aerial Ag 1098
Aerial Ag 1102
Aerial Ag 1104
Aerial Ag 1114
Aerial Ag 1140
Aerial Ag 1141
Aerial Ag 1142
Aerial Ag 1146
Aerial Ag 1152
Aerial Ag 1154
Aerial Ag 1161
Aerial Ag 1162
Aerial Ag 1163
Aerial Ag 1164
Aerial Ag 1169
Aerial Ag 1170
Aerial 

### show map of last pipeline image

In [43]:
x = orthColl.size()
print(x.getInfo())

1094


In [28]:
# view display of image with tx filtered Region
# print(display(aImg))
# print(aImg.geometry)
display(aImg.visualize())
# aImg.toDictionary()

### Try single, definite image export

In [54]:
i = 7249
lat = MercNGPipes.centroids[i:i+1].squeeze().y
lon = MercNGPipes.centroids[i:i+1].squeeze().x
# bbox args: west, south, east, north
bbox = ee.Geometry.BBox(lon-FHmeters,lat-FHmeters,lon+FHmeters,lat+FHmeters)

# get image(s) within box
aList = aerColl.select('R','G','B','N').filterDate(i_date,f_date).filterBounds(bbox)
aImg = ee.Image(aList.toList(100).get(1))

#### export (if following map works)

In [55]:
task = ee.batch.Export.image.toCloudStorage(aImg,
                                description=f"pipeline{i}_testingCRS",  # put part of pipeline here too
                                # ^ put desired name for the task & file in cloud storage
                                bucket = "test_export_bucket0",  # should be gee_image_exports
                                region=bbox,  # I wonder if filtering by box first works too
                                #maxPixels=1500000000  
                                )
task.start()

NameError: name 'projection' is not defined

In [53]:
mapImg(aImg,lon,lat)

# Learning the Functions 

In [4]:
print(len(MercNGPipes))
MercNGPipes.centroids


7259


0        POINT (-94.60046 29.66084)
1        POINT (-94.60381 29.66123)
2        POINT (-94.57724 29.58803)
3        POINT (-94.55632 29.67362)
4        POINT (-93.96192 29.94025)
                   ...             
7254    POINT (-101.21855 36.47476)
7255    POINT (-101.33197 36.34384)
7256    POINT (-101.44024 36.27811)
7257    POINT (-101.27388 36.38727)
7258    POINT (-101.49564 35.93603)
Name: centroids, Length: 7259, dtype: geometry

In [12]:
# ngPipes.centroids
i = 2500
lat1 = MercNGPipes.centroids[i:i+1].squeeze().y
lon1 = MercNGPipes.centroids[i:i+1].squeeze().x
bbox = MercNGPipes.geometry[i:i+1].squeeze().boundary # attempt



print(lat1,lon1)
print(type(bbox))

# print(ngPipes.beg[:1].squeeze())

32.188836019134584 -94.82044792461977
<class 'shapely.geometry.multipoint.MultiPoint'>


##### view satellite image

In [36]:
Map = folium.Map()
Map.setOptions()

Map.setCenter(lon1,lat1) # coordinates of poi
# Map.addLayer(tx, {'color': '#FECA1E', 'fillColor': '#4c4cff'})
Map.addLayer(ee.Image(img), {}, 'default color composite')


Map.setControlVisibility()
Map


 ## Try [aerial photography](https://developers.google.com/earth-engine/datasets/tags/highres)

In [9]:
aerColl = ee.ImageCollection("USDA/NAIP/DOQQ")
# i_date, f_date, & tx (bbox) defined above
alst = aerColl.select('R','G','B','N').filterDate(i_date,f_date).filterBounds(tx)

aimg = alst.toList(100).get(5)
display(aimg)

In [39]:
Map = folium.Map()
Map.setOptions()

Map.centerObject(lon1, lat1) # coordinates of poi
# Map.addLayer(tx, {'color': '#FECA1E', 'fillColor': '#4c4cff'})
Map.addLayer(ee.Image(aimg), {}, 'default color composite')

Map.setControlVisibility()
Map

still not seeing it

## Export an image

[documentation on exporting image function](https://developers.google.com/earth-engine/apidocs/export-image-tocloudstorage) (with links to drive, etc.)

In [34]:
ee.Authenticate()
ee.Initialize(project="pipeline-elevation-project")


Successfully saved authorization token.


In [37]:
task = ee.batch.Export.image.toCloudStorage(ee.Image(aimg),
                            description="testUSDAExport",  # optional
                            # ^ put desired name for the task & file in cloud storage
                            bucket = "test_export_bucket0",
                            maxPixels=1500000000  
                            )

# optional
# ^ for actual storages, use 'gee_image_exports'
'''
# other useful options
fileNamePrefix = "landsat9/",  # optional, put path if necessary here
dimensions = "1920x1080",  # optional, "WIDTHxHEIGHT"; unsure if this is useful
region = ee.Geometry.BBox(-122.24, 37.13, -122.11, 37.20),  # optional, current example is unknown region
scale = 1000,  # resolution in meters per pixel
# maxPixels, crs, and crsTransform (dependent on crs) might be useful
'''

task.start()

This runs quickly, but the actual task takes a long time to finish.<br><br>Clipping the export with `region` would be helpful. *--beginnings to this idea can be found with bbox above & [this link](https://gis.stackexchange.com/questions/439924/convert-local-file-shp-csv-into-earth-engine-ee-object)--*